# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 4 2022 9:20AM,243432,10,Stone8355928,Acute Outpatient Solutions,Released
1,Aug 4 2022 9:08AM,243429,20,8359842,"ACI Healthcare USA, Inc.",Executing
2,Aug 4 2022 9:04AM,243420,20,8359836,"ACI Healthcare USA, Inc.",Released
3,Aug 4 2022 8:57AM,243428,16,8353401,Sartorius Bioprocess Solutions,Released
4,Aug 4 2022 8:57AM,243428,16,8356073,Sartorius Bioprocess Solutions,Released
5,Aug 4 2022 8:38AM,243427,10,Enova-10414,Emerginnova,Executing
6,Aug 4 2022 8:38AM,243427,10,Enova-10415,Emerginnova,Released
7,Aug 4 2022 8:38AM,243427,10,Enova-10416,Emerginnova,Executing
8,Aug 4 2022 8:38AM,243427,10,Enova-10417,Emerginnova,Executing
9,Aug 4 2022 8:38AM,243427,10,Enova-10418,Emerginnova,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
17,243427,Executing,3
18,243427,Released,2
19,243428,Released,2
20,243429,Executing,1
21,243432,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
243425,NaN,2.0
243427,3.0,2.0
243428,NaN,2.0
243429,1.0,NaN
243432,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
243355,2.0,1.0
243396,0.0,5.0
243403,8.0,5.0
243414,0.0,1.0
243415,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
243355,2,1
243396,0,5
243403,8,5
243414,0,1
243415,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,243355,2,1
1,243396,0,5
2,243403,8,5
3,243414,0,1
4,243415,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,243355,2,1
1,243396,,5
2,243403,8,5
3,243414,,1
4,243415,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 4 2022 9:20AM,243432,10,Acute Outpatient Solutions
1,Aug 4 2022 9:08AM,243429,20,"ACI Healthcare USA, Inc."
2,Aug 4 2022 9:04AM,243420,20,"ACI Healthcare USA, Inc."
3,Aug 4 2022 8:57AM,243428,16,Sartorius Bioprocess Solutions
5,Aug 4 2022 8:38AM,243427,10,Emerginnova
10,Aug 4 2022 8:33AM,243425,10,"Senseonics, Incorporated"
12,Aug 4 2022 8:32AM,243424,10,"Methapharm, Inc."
17,Aug 4 2022 8:32AM,243423,10,"Methapharm, Inc."
18,Aug 4 2022 8:30AM,243422,10,ISDIN Corporation
31,Aug 4 2022 8:12AM,243419,19,ACG North America LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 4 2022 9:20AM,243432,10,Acute Outpatient Solutions,,1
1,Aug 4 2022 9:08AM,243429,20,"ACI Healthcare USA, Inc.",1,
2,Aug 4 2022 9:04AM,243420,20,"ACI Healthcare USA, Inc.",,1
3,Aug 4 2022 8:57AM,243428,16,Sartorius Bioprocess Solutions,,2
4,Aug 4 2022 8:38AM,243427,10,Emerginnova,3,2
5,Aug 4 2022 8:33AM,243425,10,"Senseonics, Incorporated",,2
6,Aug 4 2022 8:32AM,243424,10,"Methapharm, Inc.",,5
7,Aug 4 2022 8:32AM,243423,10,"Methapharm, Inc.",,1
8,Aug 4 2022 8:30AM,243422,10,ISDIN Corporation,,13
9,Aug 4 2022 8:12AM,243419,19,ACG North America LLC,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 4 2022 9:20AM,243432,10,Acute Outpatient Solutions,1,
1,Aug 4 2022 9:08AM,243429,20,"ACI Healthcare USA, Inc.",,1
2,Aug 4 2022 9:04AM,243420,20,"ACI Healthcare USA, Inc.",1,
3,Aug 4 2022 8:57AM,243428,16,Sartorius Bioprocess Solutions,2,
4,Aug 4 2022 8:38AM,243427,10,Emerginnova,2,3
5,Aug 4 2022 8:33AM,243425,10,"Senseonics, Incorporated",2,
6,Aug 4 2022 8:32AM,243424,10,"Methapharm, Inc.",5,
7,Aug 4 2022 8:32AM,243423,10,"Methapharm, Inc.",1,
8,Aug 4 2022 8:30AM,243422,10,ISDIN Corporation,13,
9,Aug 4 2022 8:12AM,243419,19,ACG North America LLC,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 4 2022 9:20AM,243432,10,Acute Outpatient Solutions,1,
1,Aug 4 2022 9:08AM,243429,20,"ACI Healthcare USA, Inc.",,1
2,Aug 4 2022 9:04AM,243420,20,"ACI Healthcare USA, Inc.",1,
3,Aug 4 2022 8:57AM,243428,16,Sartorius Bioprocess Solutions,2,
4,Aug 4 2022 8:38AM,243427,10,Emerginnova,2,3


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 4 2022 9:20AM,243432,10,Acute Outpatient Solutions,1.0,NaN
1,Aug 4 2022 9:08AM,243429,20,"ACI Healthcare USA, Inc.",NaN,1.0
2,Aug 4 2022 9:04AM,243420,20,"ACI Healthcare USA, Inc.",1.0,NaN
3,Aug 4 2022 8:57AM,243428,16,Sartorius Bioprocess Solutions,2.0,NaN
4,Aug 4 2022 8:38AM,243427,10,Emerginnova,2.0,3.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 4 2022 9:20AM,243432,10,Acute Outpatient Solutions,1.0,0.0
1,Aug 4 2022 9:08AM,243429,20,"ACI Healthcare USA, Inc.",0.0,1.0
2,Aug 4 2022 9:04AM,243420,20,"ACI Healthcare USA, Inc.",1.0,0.0
3,Aug 4 2022 8:57AM,243428,16,Sartorius Bioprocess Solutions,2.0,0.0
4,Aug 4 2022 8:38AM,243427,10,Emerginnova,2.0,3.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1947364,30.0,3.0
16,243428,2.0,0.0
19,1217025,46.0,16.0
20,730252,6.0,9.0
21,243414,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1947364,30.0,3.0
1,16,243428,2.0,0.0
2,19,1217025,46.0,16.0
3,20,730252,6.0,9.0
4,21,243414,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,30.0,3.0
1,16,2.0,0.0
2,19,46.0,16.0
3,20,6.0,9.0
4,21,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,30.0
1,16,Released,2.0
2,19,Released,46.0
3,20,Released,6.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20,21
Status,,,,,
Executing,3.0,0.0,16.0,9.0,0.0
Released,30.0,2.0,46.0,6.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20,21
0,Executing,3.0,0.0,16.0,9.0,0.0
1,Released,30.0,2.0,46.0,6.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20,21
0,Executing,3.0,0.0,16.0,9.0,0.0
1,Released,30.0,2.0,46.0,6.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()